#   Importing dependencies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cufflinks as cf
cf.go_offline()

Reading data

In [ ]:
data = pd.read_csv('./spam.csv', encoding='latin-1')

gave utf-8 codec error so included encoding parameter

In [ ]:
data.head()

Removing unncessary columns

In [ ]:
data = data[['v1', 'v2']]

In [ ]:
data.head()

Renaming

In [ ]:
data.rename(columns={'v1': 'target', 'v2': 'text'}, inplace=True)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

#   Distribution of data

In [ ]:
sns.countplot(x = data['target'])

We can see that the data distribution is not balanced

In [ ]:
data.isnull().sum()

No null values

In [ ]:
data.shape

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

Now creating a length column for each text

In [ ]:
data["length"] = data["text"].apply(lambda x: len(x))

In [ ]:
data.head()

In [ ]:
g = data.pivot(columns="target", values="length").iplot(kind="box", asFigure=True, boxpoints='outliers')
g.update_layout(
    width=800,
    height=600,
)

Length of Ham messages lie between 2-174, averaging at 52.
Length of spam messages lie between 13-224, averaging at 149.
So we figure that spam messages generally are longer.

#   Pre-processing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

In [ ]:
data.loc[3008].text

In [ ]:
text = 'Update_Now - 12Mths Half Price Orange line rental: 400mins...Call MobileUpd8 on 08000839402 or call2optout=J5Q'
text = text.lower()
text

Removing punctuations and numbers

In [ ]:
text = re.sub(r'[^a-zA-Z\s]', ' ', text)
text

In [ ]:
words = nltk.word_tokenize(text)
stopwords = stopwords.words("english")
words = [word for word in words if word not in stopwords]
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word, pos="v") for word in words]
" ".join(words)

In [ ]:
def pre_process(text):
    
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    words = nltk.word_tokenize(text)
    stopwords = stopwords.words("english")
    words = [word for word in words if word not in stopwords]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, pos="v") for word in words]
    
    text = " ".join(words)
    
    return text

In [ ]:
data["text"] = data["text"].apply(lambda x: pre_process(x))

In [ ]:
data.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split
X = data["text"]
y = data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [ ]:
bow = CountVectorizer()
X_train_vec = bow.fit_transform(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# train a Logistic Regression Model
model = LogisticRegression()

In [ ]:
model.fit(X_train_vec, y_train)

In [ ]:
X_test_vec = bow.transform(X_test)

In [ ]:
y_pred = model.predict(X_test_vec)

In [ ]:
y_pred

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

import seaborn as sns
sns.heatmap(cm, annot=True)

In [ ]:
cm

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)